### Tasks:

- Look for videos for people falling (Slipfall data) and annotate as much images as possible.

### NOTES:
1. I downloaded 4K video downloader to download the videos from youtube (I used two youtube videos and the rest were received from Cristophe via [google drive](https://drive.google.com/drive/folders/16VwyMpNnRhaxvbmDiCZt6ZzPrAfsfa3c).
2. I installed labelImg with:
> pip3 install labelImg <br>
3. Using the following command,I annotated a total of 608 objects (608 xmls).
> brew install ffmpeg <br>
> ffmpeg -i Video1.mp4 -vf fps=5 img-%d.jpg <br>

4. I renamed the xml files in each directory based on the video name because i need to have them all in one directory at the end to parse them, see next cell for bash command.

5. I received a script from Joseph (count_objects.py) to count how many "Walking, Standing, Running,Bending,Falling,Fallen" are there in the xml files.


**Run the xml parser code as python3 count_objects.py**

In [ ]:
for file in *.xml; do mv -- "$file" "${file%.xml}video2.xml";done

Total number of gestures obtained from both me and Youssef (initial number):<br>

- Standing: 841
- Walking: 462
- Fallen: 695
- Falling: 312
- Bending: 404
- Laying-down: 148
- Getting-up 121
- Sitting: 98
- Running: 48

### CROP THE IMAGES BASED ON THE COORDINATES FROM XML FILE

In [2]:
import numpy as np 
import bs4
from bs4 import BeautifulSoup 
import lxml
from PIL import Image 
import os
import cv2
import glob

In [96]:
#labelimg saves the images in PASCALVoc (i.e. xml format)
xml_path = '/Volumes/LaCie/videos/TBS-FallData/'

dirs = glob.glob(xml_path+'/*')

subdirs = ['Walking','Standing', 'Falling', 'Fallen','Bending', 
           'Getting_up','Laying_down','Running','Sitting']

for subdir in subdirs:
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    else:
        os.system('rm -rf %s/*' % subdir)

for fold in dirs:
    files = glob.glob(fold + '/*')
    for file in files:
        #images are saved in png format
        if '.png' in file:
            im_path = file
            #take image name
            im = im_path.split('/')[-1]
            path_noIm = im_path.split('/')[:6]
            path_noIm = '/'.join(path_noIm) 
            directory = im.split('.')[0] 
        #take xml annotations files
        elif '.xml' in file:
            file = open(file, 'r')
            content = file.readlines()
#       Combine the lines in the list into a string
            content = "".join(content)
            soup = BeautifulSoup(content, 'lxml')
            objects = soup.find('object')
            annotations = soup.find('annotation')
            imagename = annotations.find('filename').text
            #take dimensions needed to perform the crop
            xmin = int(objects.find('xmin').text)
            ymin = int(objects.find('ymin').text)
            xmax = int(objects.find('xmax').text)
            ymax = int(objects.find('ymax').text)
            #save the gesture name
            name = objects.find('name').text
            if name == subdirs[0]:
                subdir = subdirs[0]
            elif name == subdirs[1]:
                subdir = subdirs[1]
            elif name == subdirs[2]:
                subdir = subdirs[2]
            elif name == subdirs[3]:
                subdir = subdirs[3]
            elif name == subdirs[4]:
                subdir = subdirs[4]
            elif name == subdirs[5]:
                subdir = subdirs[5]
            elif name == subdirs[6]:
                subdir = subdirs[6]
            elif name == subdirs[7]:
                subdir = subdirs[7]
            elif name == subdirs[8]:
                subdir = subdirs[8]
            #save image name
            if imagename == im:
                image = cv2.imread(im_path)
                cropped = image[ymin:ymax,xmin:xmax]
                cv2.imwrite("{0}/{1}".format(subdir,im), cropped)
                
                
                
            
                
            





Since the videos i annotated (compilation, Grocery2) were downloaded with normal quality (640x360) , I redownloaded them with youtube-dl and split them again. The quality was much better (compilation: 1280 × 720 , Grocery2: 1920 × 1080 )

> brew install youtube-dl 


> youtube-dl --version


> youtube-dl 'url'

### Merging directories + give labels to each class
#### giving labels to ensure that when i merge them no images replace each other

In [ ]:
#adding labels to names
for file in *.jpg; do mv -- "$file" "${file%.jpg}_1.jpg";done

bending:1 <br>
falling:2 <br>
fallen:3 <br>
laying_down:4<br>
sitting:5<br>     
walking:6 <br>
getting_up:7<br>
running:8 <br>
standing:9 <br>

I split the directories to two clasees : danger (fallen and falling) and safe (all other classes). <br>
**safe**: 2121
**danger**: 2006

### For models training, check google colab notebooks!